In [9]:
import pandas as pd
import numpy as np
import base64
from io import BytesIO
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast
import torch

In [10]:
model = BloomForCausalLM.from_pretrained("bigscience/bloom-1b1")
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-1b1")

In [14]:
prompt = """ 
    Q :Bonjour
    A :Bonjour, en quoi puisse-je t'aider ?
    Q :Qui es-tu ?
    A :Je suis un chatbot créé par Sidali DIB
    Q :De quoi t'es capable ?
    A :Je suis capable de répondre à des questions de manière pertinentes dans les plus brefs délais
    <conversation history>
    Q :<question>
    """

In [17]:
question = "Combien coute un iphone ?"
conversation_history = ""
result_length = 100

In [20]:
prompt = prompt.replace("<conversation history>", conversation_history)
prompt = prompt.replace("<question>", question)

inputs = tokenizer(prompt, return_tensors="pt")

In [22]:
# Greedy Search
answer = tokenizer.decode(model.generate(inputs["input_ids"], max_new_tokens=result_length, repetition_penalty=1)[0])
answer = answer[len(prompt):].split("Q :")[0]

In [13]:
# Beam Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_new_tokens=result_length, 
                       num_beams=2, 
                       no_repeat_ngram_size=2,
                       early_stopping=True
                      )[0]))

Combien coute un iphone? Le prix d’un iphone varie en fonction de la marque, de l’emplacement et du modèle. Voici un comparatif des prix des différents modèles d’iphone.</s>


In [1]:
# Sampling Top-k + Top-p
reponse = tokenizer.decode(model.generate(inputs["input_ids"],
                       max_new_tokens=result_length, 
                       do_sample=True, 
                       top_k=1,
                       top_p=0.9,
                       temperature=0.9, 
                       repetition_penalty = 2.0 
                      )[0])
print(reponse)

NameError: name 'tokenizer' is not defined

In [18]:
reponse.replace(prompt,'')

' Les prix des smartphones sont en constante évolution. En effet, les fabricants de téléphones ont tendance à proposer plus ou moins cher leurs produits selon le marché dans lequel ils se trouvent.\nLes modèles haut-de-gamme comme l’iPhone Xs Max et la dernière version d’iOS 12 peuvent coûter jusqu’à 1 000 euros tandis que ceux qui ne dépassent pas ce seuil coûtant entre 500 € pour une tablette Android 8 Go au minimum (iPad Air) jusqu’aux 2 300€ avec'